# fct_example notebook
AWS Glue notebook fixture for doxcer tests (bronze to gold flow).

In [ ]:
%glue_version 4.0
%worker_type G.1X
%number_of_workers 2

In [ ]:
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from pyspark.sql import functions as F

sc = SparkContext.getOrCreate()
glue_context = GlueContext(sc)
spark = glue_context.spark_session

In [ ]:
p_load_date = "2026-02-13"
p_force_full_load = False
target_table = "gold.fct_daily_customer_sales"

In [ ]:
bronze_orders_df = spark.createDataFrame(
    [
        ("1001", "C001", "2026-02-13", 125.50, "completed"),
        ("1002", "C002", "2026-02-13", 20.00, "cancelled"),
        ("1003", "C001", "2026-02-13", 88.00, "completed")
    ],
    ["order_id", "customer_id", "order_date", "order_amount", "status"]
)

gold_fct_daily_customer_sales_df = (
    bronze_orders_df
    .filter(F.col("status") == "completed")
    .groupBy("order_date", "customer_id")
    .agg(
        F.sum("order_amount").alias("daily_revenue"),
        F.count("*").alias("order_count")
    )
)

gold_fct_daily_customer_sales_df.show()

In [ ]:
(
    gold_fct_daily_customer_sales_df
    .write
    .mode("overwrite")
    .parquet("s3://doxcer-example-gold/fct_daily_customer_sales/")
)

print(f"Wrote {target_table} for load date {p_load_date}")